In [ ]:
from keras.datasets.cifar10 import load_data
import matplotlib.pyplot as plt

In [ ]:
(x_train,y_train),(x_test,y_test) = load_data()
x_train.shape

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,LeakyReLU,Dropout,Reshape,Conv2DTranspose
from keras.optimizers import Adam
import numpy as np
from keras.utils import plot_model

**Discriminator model:**

In [ ]:
def discriminator(in_shape = (32,32,3)):

  model = Sequential()

  model.add(Conv2D(64,(3,3),padding= "same" , input_shape = in_shape))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(128,(3,3),padding= "same" , strides=(2,2)))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(128,(3,3),padding= "same" , strides=(2,2)))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(256,(3,3),padding= "same" , strides=(2,2)))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1,activation='sigmoid'))

  opt = Adam(learning_rate=0.0002,beta_1=0.5)

  model.compile(optimizer = opt, loss = 'binary_crossentropy',metrics = ['accuracy'])

  return model

In [ ]:
model = discriminator()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        640       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 28, 28, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 14, 14, 128)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 128)         147584    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 7, 7, 128)         0         
                                                                 
 conv2d_4 (Conv2D)           (None, 4, 4, 256)        

In [ ]:
def load_real_sample():
  (x_train,_),(_,_) = load_data()
  x = x_train.astype('float32')
  x = (x-127.5)/127.5
  return x

x = load_real_sample()
# x.shape

In [ ]:
def generate_real_sample(dataset,n_sample):
  ix = np.random.randint(0,dataset.shape[0],n_sample)
  x = dataset[ix]
  y = np.ones((n_sample,1))
  return x,y

In [ ]:
def generate_fake_sample(n_sample):
  x = np.random.rand(32 * 32 * 3 * n_sample)
  x = -1 + x * 2
  x = x.reshape((n_sample,32,32,3))
  y = np.zeros((n_sample,1))
  return x,y

x,y = generate_fake_sample(64)

In [ ]:
def train_discriminator(model,dataset,n_iter=20,n_batch=128):
  half_batch = n_batch//2

  for i in range(n_iter):
    x_real,y_real = generate_real_sample(dataset,half_batch)
    _,real_acc = model.train_on_batch(x_real,y_real)
    x_fake,y_fake = generate_fake_sample(half_batch)
    _,fake_acc = model.train_on_batch(x_fake,y_fake)
    print(f'{i+1} . real = {real_acc * 100}% , fake = {fake_acc * 100}% ')

In [ ]:
model = discriminator()
dataset = load_real_sample()

train_discriminator(model,dataset)

**Generator Model:**

In [ ]:
def generator(latent_dim):

  model = Sequential()
  n_nodes = 256*4*4

  model.add(Dense(n_nodes,input_dim = latent_dim))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Reshape((4,4,256)))

  model.add(Conv2DTranspose(128,(4,4),strides=(2,2),padding = 'same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(128,(4,4),strides=(2,2),padding = 'same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(128,(4,4),strides=(2,2),padding = 'same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(3,(3,3),activation='tanh',padding='same'))

  return model

In [ ]:
model1 = generator(100)
model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 4096)              413696    
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 4096)              0         
                                                                 
 reshape (Reshape)           (None, 4, 4, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 8, 8, 128)         524416    
 anspose)                                                        
                                                                 
 leaky_re_lu_17 (LeakyReLU)  (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 16, 16, 128)       262272    
 Transpose)                                           

In [ ]:
def generate_latent_points(latent_dim,n_sample):

  x_input = np.random.randn(latent_dim * n_sample)
  x_input = x_input.reshape(n_sample,latent_dim)
  return x_input

In [ ]:
def generate_fake_sample_gen(g_model,latent_dim,n_sample):
  x_input = generate_latent_points(latent_dim,n_sample)
  x = g_model.predict(x_input)
  y = np.zeros((n_sample,1))
  return x,y

In [ ]:
model = generator(100)
x,y = generate_fake_sample_gen(model,100,49)

**GAN:**

In [ ]:
def gan(g_model,d_model):
  d_model.trainable = False
  model = Sequential()
  model.add(g_model)
  model.add(d_model)
  opt = Adam(learning_rate = 0.0002,beta_1 = 0.5)
  model.compile(optimizer = opt,loss = 'binary_crossentropy')
  return model

In [ ]:
d_model = discriminator()
g_model = generator(100)
model = gan(g_model,d_model)

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_10 (Sequential)  (None, 32, 32, 3)         1466115   
                                                                 
 sequential_9 (Sequential)   (None, 1)                 522497    
                                                                 
Total params: 1988612 (7.59 MB)
Trainable params: 1466115 (5.59 MB)
Non-trainable params: 522497 (1.99 MB)
_________________________________________________________________


In [ ]:
def summerize_function(i,g_model,d_model,dataset,latent_dim,n_samples = 150):

  x_real,y_real = generate_real_sample(dataset,n_samples)
  _,acc_real = d_model.evaluate(x_real,y_real)

  x_fake,y_fake = generate_fake_sample_gen(g_model,latent_dim,n_samples)
  _,acc_fake = d_model.evaluate(x_fake,y_fake)

  print(f'Discriminator Accuracy: Real = {acc_real} , Fake = {acc_fake}')
  save_plot(x_fake,i)
  filename = 'model_%03d.h5' % (i+1)
  g_model.save(filename)

In [ ]:
def train(g_model,d_model,gan_model,dataset,latent_dim,n_epochs=20,n_batch=128):

  batch_per_epoch = dataset.shape[0]//n_batch
  half_batch = n_batch//2

  for i in range(n_epochs):
    for j in range(batch_per_epoch):

      x_real,y_real = generate_real_sample(dataset,half_batch)
      d_loss1,_ = d_model.train_on_batch(x_real,y_real)

      x_fake,y_fake = generate_fake_sample_gen(g_model,latent_dim,half_batch)
      d_loss2,_ = d_model.train_on_batch(x_fake,y_fake)

      x_gan = generate_latent_points(latent_dim,n_batch)
      y_gan = np.ones((n_batch,1))
      g_loss = gan_model.train_on_batch(x_gan,y_gan)

      print(f'{i+1}. {j+1}/{batch_per_epoch} : d1 = {d_loss1} , d2 = {d_loss2} , g = {g_loss}')

    if (i+1)%10 == 0:
      summerize_function(i,g_model,d_model,dataset,latent_dim)


In [ ]:
d_model = discriminator()
g_model = generator(100)

gan_model = gan(g_model,d_model)
train(g_model,d_model,gan_model,dataset,latent_dim=100,n_epochs=200,n_batch=128)

In [ ]:
def save_plot(example,epoch,n=7):

  example = (example + 1)/2.0

  for i in range(n * n):
    plt.subplot(n,n,i+1)
    plt.axis('off')
    plt.imshow(example[i])
  filename = 'plot_e%03d.png' % (epoch+1)

  plt.savefig(filename)
  plt.close()